## Fasttext
- word2vec은 사전에 존재하지 않는 단어를 잡아내지 못하는 단점이 있다.
- Fasttext를 통해 이러한 OOV 문제를 해결하며 준수한 성능까지 유도할 수 있을지 알아보자.

## Import

In [1]:
import pandas as pd
import numpy as np

from konlpy.tag import Mecab
from gensim.models import FastText

### Datas

In [2]:
df1 = pd.read_csv("data/maeili.csv", encoding='utf-8')
df2 = pd.read_csv("data/baby.csv", encoding='utf-8')
df3 = pd.read_csv("data/toys.csv", encoding='utf-8')
df4 = pd.read_csv("data/seoulchildcare.csv", encoding='utf-8')
df5 = pd.read_csv("data/babynews_DLC.csv", encoding='utf-8')
df6 = pd.read_csv("data/ildongfoodis.csv", encoding='utf-8')
df7 = pd.read_csv("data/momsmagazine.csv", encoding='utf-8')
df8 = pd.read_csv("data/ange.csv", encoding='utf-8')
df9 = pd.read_csv("data/babytimes.csv", encoding='utf-8')
df10 = pd.read_csv("data/daekyo.csv", encoding='utf-8')

### Concatenate Dataframe

In [3]:
df = pd.concat([df1.text, df2.text, df3.text, df4.text, df5.text,
                df6.text, df7.text, df8.text, df9.text, df10.text])
df = pd.DataFrame(df).reset_index(drop=True)
df.tail()

,text
20584,토끼띠의 단점
20585,국가영어능력평가 준비 올해부터 본격적으로 시행되는 국가영어능력평가에 대해서 소개해드...
20586,워킹맘이든 전업맘이든 아이들에게 관심과 사랑을 쏟아야 하는 것은 분명합니다 평소 아...
20587,검사결과 지민이의 강점지능은 언어지능과 음악지능였습니다 엄마는 지민이가 집에서나 학...
20588,진로상담센터 교육 컨설팅 아이들의 진로는 중·고등학생 때 뿐 아니라 초등학생때부터 ...


### Preprocessing

In [4]:
import re

def cleaningText(text):
    text = re.sub('&nbsp;|\xa0|\u200b|\r|\n|\t', ' ', text)
    text = re.sub('<.+>|【.+】|\[.+\]', ' ', text)
    text = re.sub('[-_=+,/\?;:^$.@*\"※~&%ㆍ!”“’「」『』〈〉\\‘|\(\)\{\}\[\]\<\>`\'…《》]', ' ', text)
    text = re.sub('[㈜×–～→▪‧•∘○□◇△▽▷◁☆◈◐▶▼▲◆★♦■♥️●♠🖤]', ' ', text)
    text = re.sub('[①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳]', ' ', text)
    text = re.sub('[➊❷➌❹❺❻❼❽❾❿]', ' ', text)
    text = re.sub('http.+com|http.+co|http.+kr|www.+kr|www.+com|www.+net|http.+net', ' ', text)
    text = re.sub("\s.+\s기자", '', text)
    text = re.sub('ⓒ\S+\s|㈜\S+\s|홈페이지\S+\s|전화\S+\s|대표전화\S+\s|☎\S+\s|http\S+\s', '', text)
    
    text = re.sub('\s+', ' ', text)
    text = text.strip()

    return text

In [5]:
df = df.drop(index=[952, 955, 956, 960, 962, 964, 1165, 1240, 1322])
df = df.drop_duplicates()
df = df.dropna()
df = pd.DataFrame(df).reset_index(drop=True)

In [6]:
df['text'] = df['text'].apply(cleaningText)
df = df.dropna()

### FastText 학습에 적합한 말뭉치로 정제하는 과정
- 한글을 자음, 모음 단위로 분해
- 단어 단위로 구분된 ndarray 자료구조로 정제

In [7]:
import re
from soynlp.hangle import decompose, compose

def remove_doublespace(s):
    doublespace_pattern = re.compile('\s+')
    return doublespace_pattern.sub(' ', s).strip()

def encode(s):
    def process(c):
        if c == ' ':
            return c
        jamo = decompose(c)
        # 'a' or 모음 or 자음
        if (jamo is None) or (jamo[0] == ' ') or (jamo[1] == ' '):
            return ' '
        base = jamo[0]+jamo[1]
        if jamo[2] == ' ':
            return base + '-'
        return base + jamo[2]

    s = ''.join(process(c) for c in s)
    return remove_doublespace(s).strip()

#     s = [process(c) for c in s]
#     return s

def decode(s):
    def process(t):
        assert len(t) % 3 == 0
        t_ = t.replace('-', ' ')
        chars = [tuple(t_[3*i:3*(i+1)]) for i in range(len(t_)//3)]
        recovered = [compose(*char) for char in chars]
        recovered = ''.join(recovered)
        return recovered

    return ' '.join(process(t) for t in s.split())

#### Test

In [8]:
s = '나의 야근을 알까? 흐헤헤 흐헤헤 아이고난1 아이고난2'
print(encode(s))
print(decode(encode(s)))

ㄴㅏ-ㅇㅢ- ㅇㅑ-ㄱㅡㄴㅇㅡㄹ ㅇㅏㄹㄲㅏ- ㅎㅡ-ㅎㅔ-ㅎㅔ- ㅎㅡ-ㅎㅔ-ㅎㅔ- ㅇㅏ-ㅇㅣ-ㄱㅗ-ㄴㅏㄴ ㅇㅏ-ㅇㅣ-ㄱㅗ-ㄴㅏㄴ
나의 야근을 알까 흐헤헤 흐헤헤 아이고난 아이고난


In [9]:
df['encoded'] = df['text'].apply(encode)
df

,text,encoded
0,6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 ...,ㅅㅏㄹ ㄴㅏㅁㅇㅏ- ㅇㅣㅂㄴㅣ-ㄷㅏ- ㅎㅚ-ㅅㅏ- ㅂㅗㄱㅈㅣㄱ ㅎㅜ- ㅅㅏㄹㅂㅜ-ㅌ...
1,요즘 들어 아빠가 아이에게 다가가려고 하면 엄마한테 갈 거야 하면서 엄마를 유독 찾...,ㅇㅛ-ㅈㅡㅁ ㄷㅡㄹㅇㅓ- ㅇㅏ-ㅃㅏ-ㄱㅏ- ㅇㅏ-ㅇㅣ-ㅇㅔ-ㄱㅔ- ㄷㅏ-ㄱㅏ-ㄱㅏ-...
2,둘째가 태어나면 첫째가 힘들 거라는 건 예상하고 있었지만 툭하면 삐치고 울고 좀처럼...,ㄷㅜㄹㅉㅐ-ㄱㅏ- ㅌㅐ-ㅇㅓ-ㄴㅏ-ㅁㅕㄴ ㅊㅓㅅㅉㅐ-ㄱㅏ- ㅎㅣㅁㄷㅡㄹ ㄱㅓ-ㄹㅏ-...
3,20개월 남아가 있고 임신 8개월차로 두 아기의 엄마입니다 첫 아이는 잠자리 독립을...,ㄱㅐ-ㅇㅝㄹ ㄴㅏㅁㅇㅏ-ㄱㅏ- ㅇㅣㅆㄱㅗ- ㅇㅣㅁㅅㅣㄴ ㄱㅐ-ㅇㅝㄹㅊㅏ-ㄹㅗ- ㄷㅜ...
4,14개월 남자 아기인데 심하게 깨무는 버릇이 있어요 전에는 졸릴 때 주로 물곤 했는...,ㄱㅐ-ㅇㅝㄹ ㄴㅏㅁㅈㅏ- ㅇㅏ-ㄱㅣ-ㅇㅣㄴㄷㅔ- ㅅㅣㅁㅎㅏ-ㄱㅔ- ㄲㅐ-ㅁㅜ-ㄴㅡㄴ...
...,...,...
19502,토끼띠의 단점,ㅌㅗ-ㄲㅣ-ㄸㅣ-ㅇㅢ- ㄷㅏㄴㅈㅓㅁ
19503,국가영어능력평가 준비 올해부터 본격적으로 시행되는 국가영어능력평가에 대해서 소개해드...,ㄱㅜㄱㄱㅏ-ㅇㅕㅇㅇㅓ-ㄴㅡㅇㄹㅕㄱㅍㅕㅇㄱㅏ- ㅈㅜㄴㅂㅣ- ㅇㅗㄹㅎㅐ-ㅂㅜ-ㅌㅓ- ㅂ...
19504,워킹맘이든 전업맘이든 아이들에게 관심과 사랑을 쏟아야 하는 것은 분명합니다 평소 아...,ㅇㅝ-ㅋㅣㅇㅁㅏㅁㅇㅣ-ㄷㅡㄴ ㅈㅓㄴㅇㅓㅂㅁㅏㅁㅇㅣ-ㄷㅡㄴ ㅇㅏ-ㅇㅣ-ㄷㅡㄹㅇㅔ-ㄱㅔ...
19505,검사결과 지민이의 강점지능은 언어지능과 음악지능였습니다 엄마는 지민이가 집에서나 학...,ㄱㅓㅁㅅㅏ-ㄱㅕㄹㄱㅘ- ㅈㅣ-ㅁㅣㄴㅇㅣ-ㅇㅢ- ㄱㅏㅇㅈㅓㅁㅈㅣ-ㄴㅡㅇㅇㅡㄴ ㅇㅓㄴㅇ...


In [10]:
encoded_corpus = np.array(df.encoded)
type(encoded_corpus), len(encoded_corpus)

(numpy.ndarray, 19507)

### Train

In [20]:
%%time

class FastTextCorpus:
    def __init__(self, corpus):
        self.corpus = corpus
        self.n_iter = 0
    def __iter__(self):
        for i, doc in enumerate(self.corpus):
            if i % 500 == 0 and i > 0:
                print('\riter = {}, sents = {} ...'.format(self.n_iter, i), end='')
            yield doc.split()
        self.n_iter += 1
        print('\riter = {}, docs = {} done{}'.format(self.n_iter, i, ' '*20))
        
corpus = FastTextCorpus(encoded_corpus)
fasttext_model = FastText(
    corpus,
    size=100,
    seed=191212,
    workers=6,
    window=3,
    min_count=6,
    min_n=2,
    max_n=6,
)

iter = 1, docs = 19506 done                    
iter = 2, docs = 19506 done                    
iter = 3, docs = 19506 done                    
iter = 4, docs = 19506 done                    
iter = 5, docs = 19506 done                    
iter = 6, docs = 19506 done                    
CPU times: user 5min 56s, sys: 1.64 s, total: 5min 58s
Wall time: 1min 21s


In [30]:
fasttext_model.save('data/tokmom_fasttext.model')

In [21]:
def most_similar(query, topn=10):
    query_ = encode(query)
    similars = fasttext_model.wv.most_similar(query_, topn=topn)
    similars = [(decode(word), sim) for word, sim in similars]
    return similars

In [22]:
most_similar("여자아이")

[('남자아이', 0.9075375199317932),
 ('여자아이가', 0.9023966789245605),
 ('여자아이들이', 0.8702645301818848),
 ('여자아이보다', 0.86384117603302),
 ('여자아이는', 0.8608008623123169),
 ('첫아이', 0.8579444885253906),
 ('셋째아이', 0.856373131275177),
 ('경희아이', 0.8479938507080078),
 ('둘째아이', 0.8464566469192505),
 ('남자아이가', 0.8446069955825806)]

In [23]:
most_similar("여아")

[('역아', 0.8398646116256714),
 ('영아', 0.8389644622802734),
 ('아쿠아', 0.8005619645118713),
 ('여타', 0.7780101299285889),
 ('영아연축', 0.7776792049407959),
 ('여태', 0.7587387561798096),
 ('쌍생아', 0.7567449808120728),
 ('여덟', 0.755237340927124),
 ('여아가', 0.7472755908966064),
 ('쏘아', 0.7468857765197754)]

In [24]:
most_similar("남자아이")

[('여자아이', 0.9075374007225037),
 ('남자아이가', 0.8996829986572266),
 ('셋째아이', 0.8893890976905823),
 ('둘째아이', 0.8763366937637329),
 ('나이', 0.8712449073791504),
 ('첫아이', 0.8665239810943604),
 ('남자아이는', 0.8632090091705322),
 ('남자아이들이', 0.8598025441169739),
 ('남자아이보다', 0.8564349412918091),
 ('큰아이', 0.8548170328140259)]

In [25]:
most_similar("남아")

[('남겨', 0.8145788311958313),
 ('살아남아', 0.7954429984092712),
 ('남겨져', 0.7922765016555786),
 ('낳아', 0.7914118766784668),
 ('삼아', 0.7661623358726501),
 ('남녀', 0.7624686360359192),
 ('남아서', 0.7522130012512207),
 ('쌓아', 0.7472477555274963),
 ('남아가', 0.7408424615859985),
 ('녹아', 0.7408424615859985)]

In [26]:
most_similar("치아")

[('치아나', 0.8355734348297119),
 ('치아관리', 0.8103460073471069),
 ('치아건강', 0.7775462865829468),
 ('소피아', 0.7705380916595459),
 ('미아', 0.7664532661437988),
 ('아리아', 0.747617244720459),
 ('치아가', 0.741093635559082),
 ('치태', 0.7407466173171997),
 ('치아발육기', 0.7406258583068848),
 ('치매', 0.7400088310241699)]

In [27]:
most_similar("이빨")

[('이탈', 0.9184857606887817),
 ('이혼할', 0.8759535551071167),
 ('이어갈', 0.8619340658187866),
 ('이길', 0.856127142906189),
 ('이걸', 0.8507415056228638),
 ('이룰', 0.8410588502883911),
 ('이달', 0.839617908000946),
 ('이불', 0.8386311531066895),
 ('이럴', 0.8299028277397156),
 ('옛날', 0.8246774673461914)]

In [28]:
most_similar("놀이")

[('책놀이', 0.9498100280761719),
 ('공놀이', 0.9358961582183838),
 ('윷놀이', 0.9310749769210815),
 ('까꿍놀이', 0.9170355796813965),
 ('소꿉놀이', 0.9155725240707397),
 ('색칠놀이', 0.9078812003135681),
 ('놀이터가', 0.9060976505279541),
 ('음악놀이', 0.9000377058982849),
 ('촉감놀이', 0.8935685753822327),
 ('점토놀이', 0.8932490348815918)]

In [29]:
most_similar("감기")

[('열감기', 0.897197425365448),
 ('코감기', 0.884961724281311),
 ('감기약', 0.8469191193580627),
 ('염기', 0.8354255557060242),
 ('갈기', 0.8332834243774414),
 ('가볍기', 0.8328777551651001),
 ('간절기', 0.8326712250709534),
 ('감염되기', 0.8215770721435547),
 ('감기처럼', 0.8205673694610596),
 ('가습기', 0.8147245645523071)]

### 결과 확인
하이퍼 파라미터 조정을 다양하게 시도해보며 결과를 확인해봤지만, 역시 사용하기 어렵겠다는 결론을 내렸다. 말뭉치가 부족한 것이 원인이 아닐까?

---

## 못내 아쉬워서 재도전
- 이번엔 자음, 모음 단위가 아닌 글자 단위로 해보자
- 간단하게 Mecab으로 tokenizing한 말뭉치를 학습으로 사용

In [31]:
from konlpy.tag import Mecab

In [32]:
def tokenize(text):
    tokenizer = Mecab()
    text = tokenizer.morphs(text)
    text = ' '.join(text)
    return text

In [33]:
%%time
df['tokenized'] = df['text'].apply(tokenize)
df.tail()

CPU times: user 42 s, sys: 22.6 s, total: 1min 4s
Wall time: 1min 4s


,text,encoded,tokenized
19502,토끼띠의 단점,ㅌㅗ-ㄲㅣ-ㄸㅣ-ㅇㅢ- ㄷㅏㄴㅈㅓㅁ,토끼띠 의 단점
19503,국가영어능력평가 준비 올해부터 본격적으로 시행되는 국가영어능력평가에 대해서 소개해드...,ㄱㅜㄱㄱㅏ-ㅇㅕㅇㅇㅓ-ㄴㅡㅇㄹㅕㄱㅍㅕㅇㄱㅏ- ㅈㅜㄴㅂㅣ- ㅇㅗㄹㅎㅐ-ㅂㅜ-ㅌㅓ- ㅂ...,국가 영어 능력 평가 준비 올해 부터 본격 적 으로 시행 되 는 국가 영어 능력 평...
19504,워킹맘이든 전업맘이든 아이들에게 관심과 사랑을 쏟아야 하는 것은 분명합니다 평소 아...,ㅇㅝ-ㅋㅣㅇㅁㅏㅁㅇㅣ-ㄷㅡㄴ ㅈㅓㄴㅇㅓㅂㅁㅏㅁㅇㅣ-ㄷㅡㄴ ㅇㅏ-ㅇㅣ-ㄷㅡㄹㅇㅔ-ㄱㅔ...,워킹 맘 이 든 전업 맘 이 든 아이 들 에게 관심 과 사랑 을 쏟 아야 하 는 것...
19505,검사결과 지민이의 강점지능은 언어지능과 음악지능였습니다 엄마는 지민이가 집에서나 학...,ㄱㅓㅁㅅㅏ-ㄱㅕㄹㄱㅘ- ㅈㅣ-ㅁㅣㄴㅇㅣ-ㅇㅢ- ㄱㅏㅇㅈㅓㅁㅈㅣ-ㄴㅡㅇㅇㅡㄴ ㅇㅓㄴㅇ...,검사 결과 지민 이 의 강점 지능 은 언어 지능 과 음악 지능 였 습니다 엄마 는 ...
19506,진로상담센터 교육 컨설팅 아이들의 진로는 중·고등학생 때 뿐 아니라 초등학생때부터 ...,ㅈㅣㄴㄹㅗ-ㅅㅏㅇㄷㅏㅁㅅㅔㄴㅌㅓ- ㄱㅛ-ㅇㅠㄱ ㅋㅓㄴㅅㅓㄹㅌㅣㅇ ㅇㅏ-ㅇㅣ-ㄷㅡㄹㅇ...,진로 상담 센터 교육 컨설팅 아이 들 의 진로 는 중 · 고등학생 때 뿐 아니 라 ...


In [34]:
encoded_corpus = np.array(df.tokenized)
len(encoded_corpus)

19507

In [35]:
%%time
class FastTextCorpus:
    def __init__(self, corpus):
        self.corpus = corpus
        self.n_iter = 0
    def __iter__(self):
        for i, doc in enumerate(self.corpus):
            if i % 500 == 0 and i > 0:
                print('\riter = {}, sents = {} ...'.format(self.n_iter, i), end='')
            yield doc.split()
        self.n_iter += 1
        print('\riter = {}, docs = {} done{}'.format(self.n_iter, i, ' '*20))
        
corpus = FastTextCorpus(encoded_corpus)
fasttext_token_model = FastText(
    corpus,
    size=100,
    seed=191212,
    workers=6,
    window=5,
    min_count=6,
    min_n=2,
    max_n=6,
)

iter = 1, docs = 19506 done                    
iter = 2, docs = 19506 done                    
iter = 3, docs = 19506 done                    
iter = 4, docs = 19506 done                    
iter = 5, docs = 19506 done                    
iter = 6, docs = 19506 done                    
CPU times: user 2min 39s, sys: 871 ms, total: 2min 40s
Wall time: 37.8 s


In [36]:
fasttext_token_model.save('data/tokmom_token_fasttext.model')

In [37]:
fasttext_token_model.wv.most_similar("여아")

[('남아', 0.8793524503707886),
 ('빈아', 0.8268206715583801),
 ('동남아', 0.8174921274185181),
 ('머지않아', 0.8166212439537048),
 ('승아', 0.8153604865074158),
 ('쌍생아', 0.8029741048812866),
 ('산아', 0.7987430095672607),
 ('단아', 0.7971248030662537),
 ('우량아', 0.7963160276412964),
 ('맥아', 0.7935917377471924)]

In [38]:
fasttext_token_model.wv.most_similar("여자아이")

[('여자', 0.6591677665710449),
 ('남자', 0.6501051187515259),
 ('큰아이', 0.6341002583503723),
 ('갓난아이', 0.6282140016555786),
 ('딸아이', 0.623320460319519),
 ('외동아이', 0.6097438335418701),
 ('또래', 0.5886651873588562),
 ('첫아이', 0.5864680409431458),
 ('아이', 0.5809661149978638),
 ('어린아이', 0.5537697076797485)]

In [39]:
fasttext_token_model.wv.most_similar("남아")

[('여아', 0.8793525099754333),
 ('딸아', 0.8142745494842529),
 ('동남아', 0.8132936358451843),
 ('빈아', 0.7888689041137695),
 ('우량아', 0.7881900072097778),
 ('이진아', 0.7869821786880493),
 ('머지않아', 0.7833974361419678),
 ('쌍생아', 0.7821588516235352),
 ('이은아', 0.7778501510620117),
 ('이케아', 0.7753491401672363)]

In [40]:
fasttext_token_model.wv.most_similar("남자아이")

[('남자', 0.6883581876754761),
 ('외동아이', 0.6248061656951904),
 ('갓난아이', 0.6079676151275635),
 ('큰아이', 0.59782475233078),
 ('딸아이', 0.5890188217163086),
 ('어린아이', 0.581037700176239),
 ('자기', 0.5792179703712463),
 ('어른', 0.5758233070373535),
 ('강아지', 0.5727760791778564),
 ('자아', 0.5694735050201416)]

In [41]:
fasttext_token_model.wv.most_similar("치아")

[('잇몸', 0.7881530523300171),
 ('영구치', 0.780902087688446),
 ('치태', 0.7569048404693604),
 ('치조골', 0.7496535778045654),
 ('치핵', 0.7459360957145691),
 ('치골', 0.7398999929428101),
 ('유치', 0.7325750589370728),
 ('치아우식증', 0.7163764238357544),
 ('치석', 0.7086289525032043),
 ('턱뼈', 0.6980010271072388)]

In [42]:
fasttext_token_model.wv.most_similar("이빨")

[('이삭', 0.902835488319397),
 ('이뻐', 0.8997254371643066),
 ('이놈', 0.8995668888092041),
 ('이따', 0.8974958658218384),
 ('이끼', 0.897071123123169),
 ('이착륙', 0.8884831666946411),
 ('이끈', 0.8866316080093384),
 ('이메', 0.8751682043075562),
 ('이카리', 0.8699648976325989),
 ('이랬', 0.865716814994812)]

In [52]:
fasttext_token_model.wv.most_similar("이")

[('이갈이', 0.6703658103942871),
 ('이법이', 0.6637637615203857),
 ('이앓이', 0.6623647809028625),
 ('이이', 0.6623225212097168),
 ('이고은', 0.6336269974708557),
 ('이걸로', 0.6267480254173279),
 ('이승로', 0.6218252182006836),
 ('더욱이', 0.617598295211792),
 ('이주은', 0.6152785420417786),
 ('이가', 0.6107053756713867)]

In [43]:
fasttext_token_model.wv.most_similar("놀이")

[('공놀이', 0.8616198301315308),
 ('길놀이', 0.8523542881011963),
 ('투호놀이', 0.8519386053085327),
 ('사물놀이', 0.845710277557373),
 ('놀이판', 0.8447649478912354),
 ('윷놀이', 0.8404592871665955),
 ('기차놀이', 0.8361167311668396),
 ('탈놀이', 0.8165323734283447),
 ('관자놀이', 0.8069682717323303),
 ('풍물놀이', 0.7963175773620605)]

In [44]:
fasttext_token_model.wv.most_similar("이빠ㄹ")

[('곪', 0.3686450719833374),
 ('해했', 0.3523622751235962),
 ('해졌', 0.34141188859939575),
 ('막혔', 0.3383951783180237),
 ('덧붙였', 0.33778858184814453),
 ('속상했', 0.330879271030426),
 ('불리', 0.32451584935188293),
 ('굴뚝같', 0.32420358061790466),
 ('같', 0.3142273724079132),
 ('털어놨', 0.31361037492752075)]

In [45]:
fasttext_token_model.wv.most_similar("놀ㅇ")

[('놀', 0.48708441853523254),
 ('놀랄', 0.47123098373413086),
 ('놀람', 0.47120022773742676),
 ('놀라울', 0.45342379808425903),
 ('뛰놀', 0.44498616456985474),
 ('놀이공원', 0.42896416783332825),
 ('놀랍', 0.4279695153236389),
 ('마음놓', 0.4248155355453491),
 ('놀이동산', 0.42026078701019287),
 ('놀러갔', 0.41828668117523193)]

In [46]:
fasttext_token_model.wv.most_similar("아기")

[('갓난아기', 0.8749422430992126),
 ('아이', 0.8532912731170654),
 ('아기씨', 0.8456162214279175),
 ('아기방', 0.8449693918228149),
 ('아기집', 0.8238485455513),
 ('아기과자', 0.81862473487854),
 ('갓난아이', 0.8117345571517944),
 ('아기자기', 0.8095382452011108),
 ('태아기', 0.8043420910835266),
 ('신생아기', 0.7991621494293213)]

In [47]:
fasttext_token_model.wv.most_similar("애기")

[('우기', 0.8718590140342712),
 ('솔기', 0.8703078627586365),
 ('슬기', 0.8668882250785828),
 ('뺏기', 0.8617573976516724),
 ('웃기', 0.857114315032959),
 ('쐐기', 0.8562721014022827),
 ('튕기', 0.8551444411277771),
 ('숫기', 0.8548935651779175),
 ('깍두기', 0.8546078205108643),
 ('땅기', 0.8526855707168579)]

In [48]:
fasttext_token_model.wv.most_similar("아ㄱ")

[('아테네', 0.532082200050354),
 ('아껴', 0.5209164619445801),
 ('아홉', 0.5150492191314697),
 ('아뿔싸', 0.509800374507904),
 ('아웃', 0.49506261944770813),
 ('아슬아슬', 0.4895532727241516),
 ('아저씨', 0.4893777072429657),
 ('아휴', 0.48421481251716614),
 ('아랫', 0.4805004298686981),
 ('아누', 0.4751540720462799)]

In [49]:
fasttext_token_model.wv.most_similar("찡")

[('찡긋', 0.8634486198425293),
 ('찡찡', 0.8102352619171143),
 ('깜', 0.7513332366943359),
 ('툭', 0.7431281805038452),
 ('깜빡', 0.7019097805023193),
 ('팍', 0.6868350505828857),
 ('찰칵', 0.6778855323791504),
 ('뻑뻑', 0.6770793795585632),
 ('툭툭', 0.6734962463378906),
 ('휘젓', 0.6727398633956909)]

In [50]:
fasttext_token_model.wv.most_similar("열")

[('희열', 0.8047626614570618),
 ('마찰열', 0.7351973056793213),
 ('허열', 0.7190242409706116),
 ('열혈', 0.7097938060760498),
 ('열흘', 0.694926917552948),
 ('우열', 0.6948281526565552),
 ('열독', 0.6940494775772095),
 ('균열', 0.6889867782592773),
 ('열광', 0.6877672672271729),
 ('내열', 0.6856398582458496)]

In [51]:
fasttext_token_model.wv.most_similar("육아")

[('육아종', 0.7534529566764832),
 ('육모', 0.6781485676765442),
 ('공동육아', 0.663213312625885),
 ('육아법', 0.6533761024475098),
 ('양육', 0.6316951513290405),
 ('노아', 0.6042506694793701),
 ('워킹', 0.5996135473251343),
 ('전업', 0.5879354476928711),
 ('가사', 0.5831745862960815),
 ('현아', 0.5787320733070374)]

### Review


- 이전 모델이 비해 전체적인 성능이 더 좋아진 것은 체감된다!
- 하지만 오타나 말뭉치에 존재하지 않는 단어를 입력받을 경우, 최소 error는 발생하진 않지만 성능에 대한 이슈는 여전히 존재한다 
  - 보통 오타는 자음 모음을 누락하거나 잘못 입력하는 경우에 발생하기 때문. 글자 단위로 임베딩 했을 때는 이런 부분을 잡기 어렵다.
  - 어떻게든 결과는 나오지만 fasttext만의 특색이자 장점을 못살리는 느낌.. 
- 그리고 fasttext 특성 상 "남아"의 유사어로 "동남아"가 나오는 식의 문제는 어떻게 할 수가 없다; 이것은 자음 모음 단위로 임베딩 할 경우에도 있었던 문제임
- Embedding 결과 성능을 정량적으로 비교해서 word2vec과 fasttext 둘 중 무엇이 좋을지 확실히 비교할 수 있는 방안을 찾아서 어떤 것을 활용할지 결정해야 할 듯